In [ ]:
#Install the necessary packages

!pip install yfinance
!pip install matplotlib==3.5.3
!pip install powerlaw

In [ ]:
#Standard packages
import numpy as np
import pandas as pd

#Dates
from datetime import datetime, timedelta

#Finance packages
import yfinance as yf

#Statistics
import random
import powerlaw

#Plotting packages
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from matplotlib import rcParams

rcParams["font.size"] = 20
rcParams["axes.labelsize"] = 30

rcParams["xtick.labelsize"] = 16
rcParams["ytick.labelsize"] = 16

rcParams["figure.figsize"] = (8,6)

# Exercise 1. Analysis of the distribution of returns

As you have seen in the lectures, none of the available models are able to fully characterize the properties of the returns' time series (ACF, power spectrum, distribution of returns, etc). In the last practical case we analyzed the ACF and power spectrum of the S&P 500 index, but we didnt look at its return distribution. Let's do it now!

**1. Compute the log returns of the Adjusted Close value of the index**

In [ ]:
df = yf.download("^GSPC", start=datetime(1930, 1, 1), end=datetime.today(), interval="1d", progress=False)

df["LogRet"] = #CODE

df = df.dropna()

df

**2. Split the return time-series in "positive" and "negative" and compute the probability density function (PDF) and the Complementary Cumulative Distribution Function (CCDF).**

* Which value do you obtain for the power law exponent $\alpha$?

* Is the second moment (the variance) finite?

**Clue:** *Use the powerlaw package (https://pypi.org/project/powerlaw/)*

In [ ]:
positive_returns = #CODE
negative_returns = #CODE

result_pos = powerlaw.Fit(positive_returns, xmin=1e-6)
result_neg = powerlaw.Fit(negative_returns, xmin=1e-6)

result_pos_tail = powerlaw.Fit(positive_returns, xmin=1e-2)
result_neg_tail = powerlaw.Fit(negative_returns, xmin=1e-2)

fig, ax = plt.subplot_mosaic("""AB
CD""", figsize=(8*2, 6*2))

result_pos.plot_pdf(ax=ax["A"], ls="", marker="o", label="Positive returns")
result_neg.plot_pdf(ax=ax["A"], ls="", marker="o", label="Negative returns")

result_pos.plot_ccdf(ax=ax["B"], ls="", marker="o", label="Positive returns")
result_neg.plot_ccdf(ax=ax["B"], ls="", marker="o", label="Negative returns")

result_pos_tail.plot_pdf(ax=ax["C"], ls="", marker="o", label="Positive tail")
result_neg_tail.plot_pdf(ax=ax["C"], ls="", marker="o", label="Negative tail")

result_pos_tail.plot_ccdf(ax=ax["D"], ls="", marker="o", label="Positive tail")
result_neg_tail.plot_ccdf(ax=ax["D"], ls="", marker="o", label="Negative tail")

#result_pos_tail.power_law.plot_pdf(ax=ax["D"], lw=5, color="k", label=r"$\alpha=%.2f$" % result_pos_tail.alpha)
x = np.logspace(-2, -0.5, 100)
ax["D"].plot(x, 1e-5*np.power(x, -result_pos_tail.alpha), lw=3, color="k", label=r"$\alpha=%.2f$" % result_pos_tail.alpha)

ax["A"].set_title("PDF of log returns")
ax["B"].set_title("CCDF of log returns")
ax["C"].set_title("Tail of the PDF")
ax["D"].set_title("Tail of the CCDF")

ax["A"].set_xlabel("Log Return")
ax["B"].set_xlabel("Log Return")
ax["C"].set_xlabel("Log Return")
ax["D"].set_xlabel("Log Return")

ax["A"].set_ylabel("PDF")
ax["B"].set_ylabel("CCDF")
ax["C"].set_ylabel("PDF")
ax["D"].set_ylabel("CCDF")

ax["D"].legend()

plt.subplots_adjust(wspace=0.25, hspace=0.45)

# Exercise 2. Simulation and analysis of a Lévy Flight

So it is clear that the return distribution is heavy-tailed. Mandelbrot proposed to model price dynamics using Lévy flights, a random walk process in which the step-length is a random variable distributed according to a Lévy distribution. In particular, Mandelbrot coined the Lévy flight term for step-lengths distributed according to the following distribution

\begin{equation}
P(l)=\frac{1}{l^{\alpha+1}}=l^{-(\alpha+1)}
\end{equation}

which is basically a power law.

To generate random numbers according to this distribution we need to compute the inverse of it's cummulative distribution, as you have seen in the lectures. In practice, we need to define a cutoff, or minimum value, to numerically define a power law distribution. Thus, consider the appropriate probability density function for a power law,

\begin{equation}
P(l)=\frac{\alpha}{l_{\textrm{min}}}\left(\frac{l}{l_{\textrm{min}}}\right)^{-(\alpha+1)}
\end{equation}

The cummulative distribution can be found by integrating,

\begin{equation}
P(L>l)=\int_{l}^{+\infty}P(l)=\left[-\left(\frac{l}{l_{\textrm{min}}}\right)^{-\alpha}\right]_{l}^{+\infty}=\left(\frac{l}{l_{\textrm{min}}}\right)^{-\alpha}
\end{equation}

and thus

\begin{equation}
CDF(l)=P(L<l)=1-\left(\frac{l}{l_{\textrm{min}}}\right)^{-\alpha}
\end{equation}

and by inverting it we obtain,

\begin{equation}
u=1-\left(\frac{l}{l_{\textrm{min}}}\right)^{-\alpha} \Longrightarrow l=l_{\textrm{min}}\left(1-u\right)^{-1/\alpha}
\end{equation}

So to generate random numbers distributed to a power law distribution we have to implement

\begin{equation}
l=l_{\textrm{min}}\left(1-u\right)^{-1/\alpha}
\end{equation}

with $u$ being a uniform random number between 0 and 1, $u(0,1)$.

**1. Complete the following functions to generate a Lévy flight**

**Clue:** The `random.choices(list, k=N)` method draws a random sample of N elements from the given list.

In [ ]:
def levy_steps(N, alpha, l_min):

  #CODE

def levy_flight(N, alpha, l_min, x_0):

  #CODE

**2. Analyze the distribution of steps for different values of $\alpha$**

* Use the `hist_values, bin_edges = numpy.histogram(data, bins, density::bool)` function to compute the histograms and then plot it using a scatter plot.

* Set `density`=True and use evenly spaced bins (`numpy.logspace(start, stop, N)`)

In [ ]:
#CODE

**2. Simulate a Lévy Flight**

Plot several realisations of the Lévy flight for a given value of $\alpha$

In [ ]:
#CODE

# Exercise 3. Lévy flight as model for price dynamics

Playing around with the $\alpha$ and $l_{\textrm{min}}$ parameters we can generate levy steps that resemble the observed distribution of returns.

Then, using these parameter values, we can simulate the future dynamic of the price.

In [ ]:
N = 10**6
alpha = 2.6
l_min = 0.005

steps = levy_steps(N, alpha, l_min)

hist, edges = np.histogram(steps, bins=np.logspace(-3, 2, 100), density=True);

result_pos_tail = powerlaw.Fit(positive_returns, xmin=1e-4)

fig, ax = plt.subplot_mosaic("""AB""", figsize=(8*2, 6))

result_pos_tail.plot_pdf(ax=ax["A"], ls="", marker="o", label="Positive tail")

ax["A"].scatter(edges[1:], hist, label=r"$\alpha=%.2f$" % alpha, color="r")

ax["A"].set_yscale("log")
ax["A"].set_xscale("log")

ax["B"].set_ylabel("Probability density")
ax["B"].set_xlabel("Returns")

time = np.arange(0, len(df), 1)
tf = time[-1]

N = 2*10**4
alpha = 2.6
l_min = 0.005

time_future = [df.index[-1] + timedelta(i) for i in range(N)]

steps = levy_steps(N, alpha, l_min)
directions = random.choices([-1, 1], k=N)

simulated_returns = steps*directions

ax["B"].plot(df["LogRet"])
ax["B"].plot(time_future, simulated_returns)

ax["B"].set_ylabel("Returns")

plt.subplots_adjust(wspace=0.25)

1. **Simulate price dynamics using a levy flight process.**

* Plot several realisations of the process in the same figure.

**Clue:** *Don't use the `levy_flight` function. Think that log returns are additive, so that $S(t=t1+t2)=S(0)e^{(r_1+r_2)t}$*

In [ ]:
#CODE

#Exercise 4.  Data-driven price dynamics model using the Student's t distribution

**1. Fit the distribution of returns using a t distribution**

* Plot the pdf of the fitted distribution together with the histogram of the returns.

**Clue:** *Use the `scipy.stats.t.fit` method to fit the distribution to the data. Note that this method returns a tupple of shape, loc and scale parameters for the fitted distribution. You can then use the `empirical_dist = scipy.stats.t(fited_params)` method to define the empirical distribution of returns. and then use `empirical_dist.pdf(x)` to plot the pdf of the fitted distribution.*

In [ ]:
# Your code here

**2. Use the fitted distribution to simulate future returns and future price dynamics**

Plot several realisations of the possible future dynamics of prices.

**Clue:** *You can use the `empirical_dist.rvs(N)* method tho generate random numbers according to the fitted distribution.

In [ ]:
# Your code here